In [7]:
!pip install openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [openpyxl]1/2 [openpyxl]

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [10]:
import pandas as pd
import re
path = "/workspaces/BA/datasets/"
files = ["/workspaces/BA/datasets/raw_text_data2.txt", "/workspaces/BA/datasets/raw_text_data3.txt","/workspaces/BA/datasets/raw_text_data4.txt"]
all_questions = []
ILLEGAL_CHARACTERS_RE = re.compile(r'[\000-\010]|[\013-\014]|[\016-\037]')

for f in files:
    df = pd.read_csv(f, sep="\t", encoding="latin-1")
    df['Question'] = df['Question'].astype(str).apply(
        lambda x: ILLEGAL_CHARACTERS_RE.sub("", x)
    )
    all_questions.append(df[['Question']])

# Combine them
questions_df = pd.concat(all_questions).drop_duplicates().reset_index(drop=True)

# Save to Excel
questions_df.to_excel("/workspaces/BA/datasets/all_prompts.xlsx", index=False)

print(f"Saved {len(questions_df)} unique questions from all files to all_prompts.xlsx")


Saved 2457 unique questions from all files to all_prompts.xlsx


In [1]:
import pandas as pd
import re

# Regex to clean illegal Excel characters
ILLEGAL_CHARACTERS_RE = re.compile(r'[\000-\010]|[\013-\014]|[\016-\037]')

# Step 1: Load existing Excel
questions_df = pd.read_excel("/workspaces/BA/datasets/all_prompts.xlsx")

# Step 2: Read JSONL file
jsonl_file = "/workspaces/BA/datasets/raw_text_data1.jsonl"
new_questions = pd.read_json(jsonl_file, lines=True)

# Extract only "response" column, rename to "Question"
new_questions = new_questions[['response']].rename(columns={'response': 'Question'})

# Clean illegal characters
new_questions['Question'] = new_questions['Question'].astype(str).apply(
    lambda x: ILLEGAL_CHARACTERS_RE.sub("", x)
)

# Step 3: Concat with existing
combined = pd.concat([questions_df, new_questions]).drop_duplicates().reset_index(drop=True)

# Step 4: Save back to Excel (overwrite with new combined version)
combined.to_excel("/workspaces/BA/datasets/all_prompts.xlsx", index=False)

print(f"Appended {len(new_questions)} prompts. Total now: {len(combined)} in all_prompts.xlsx")


Appended 173574 prompts. Total now: 59663 in all_prompts.xlsx
